In [10]:
from langchain_experimental.agents.agent_toolkits import create_python_agent   
from langchain_experimental.tools.python.tool import PythonREPLTool            
from langchain_ollama  import ChatOllama
from langchain import RetrievalQA, PromptTemplate, LLMChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool

ImportError: cannot import name 'RetrievalQA' from 'langchain' (/home/nitish/Documents/github/jarvis-offline/.venv/lib/python3.12/site-packages/langchain/__init__.py)

## Build an agent which has access to a Tool which writes and executes python code

In [ ]:

# Instantiate your LLM
llm = ChatOllama(model = "qwen2.5:7b", temperature=0)

# Create the REPL tool
python_repl = PythonREPLTool()

# Spin up a “python agent”
agent = create_python_agent(
    llm=llm,
    tool=python_repl,
    verbose=True,
)

# Ask it to both write & run code
result = agent.run(
    "Compute the vector norm of [3, 4, 5] and return the result.")
print("Result:", result)




> Entering new AgentExecutor chain...
I need to calculate the Euclidean norm (or vector norm) of the given vector [3, 4, 5].
Action: Python_REPL
Action Input: import math
vector = [3, 4, 5]
norm = math.sqrt(sum([x**2 for x in vector]))
print(norm)
Observation: 7.0710678118654755

Thought:I now know the final answer
Final Answer: The vector norm of [3, 4, 5] is approximately 7.071.

> Finished chain.
Result: The vector norm of [3, 4, 5] is approximately 7.071.


### Combine this with RAG

In [ ]:


# 1️⃣ Your LLM
llm = OpenAI(temperature=0)

# 2️⃣ A vectorstore + retriever for RAG
emb = OpenAIEmbeddings()
vectordb = Chroma.from_existing_collection("my_collection", embedding_function=emb)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever()
)

# 3️⃣ Wrap RAG as a Tool
rag_tool = Tool.from_function(
    rag_chain.run,
    name="rag_retriever",
    description="Use this to fetch relevant context from your document store."
)

# 4️⃣ Python REPL tool
python_repl = PythonREPLTool()

# 5️⃣ Put them both into an agent
tools = [rag_tool, Tool.from_function(python_repl.run, name="python_repl")]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# 6️⃣ Run a mixed task
prompt = """
Load the relevant data for 'quarterly earnings analysis' using rag_retriever.
Then write Python code that computes the 90-day moving average of the 'revenue' field
and return the result.
"""
print(agent.run(prompt))


Signature:
create_python_agent(
    llm: langchain_core.language_models.base.BaseLanguageModel,
    tool: langchain_experimental.tools.python.tool.PythonREPLTool,
    agent_type: langchain.agents.agent_types.AgentType = <AgentType.ZERO_SHOT_REACT_DESCRIPTION: 'zero-shot-react-description'>,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    verbose: bool = False,
    prefix: str = 'You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return "I don\'t know" as the answer.\n',
    agent_executor_kwargs: Optional[Dict[str, Any]] = None,
    **kwargs: Dict[st